In [18]:
from time import sleep
from selenium.webdriver import Opera
from selenium.webdriver.opera.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import os
import glob


In [116]:
def get_details(index):
    
    
    
    one = df_art[index]['content']

    temp_tab = one.find_all(class_='m9qz_yp mupj_ka mpof_vs _3c6dd_a2WKl')
    
    if temp_tab[2] == None:
        data[index]['brand'] = temp_tab[2]
    else:
        data[index]['brand'] = temp_tab[2].text
    
    if temp_tab[3] == None:
        data[index]['model'] = "Nieznany"
    else:
        data[index]['model'] = temp_tab[3].text
    
    one = details_page[index]
    
    one = one.find(class_ ="m9qz_yp mgn2_23 mgn2_30_s mp4t_16 mryx_0 _b1f72_28UM3")
    
    if one == None:
        data[index]['rating']= one
    else:
        data[index]['rating'] = one.text
    
    
    one = details_page[index]
    
    temp_tab = None
    temp_tab = one.find_all('h4')
    
        
    one = details_page[index]
    one = one.find('div', class_="mp0t_0a mqu1_21 mli8_k4 mgn2_13 mgmw_ag mp4t_8")

    if one == None:
        data[index]['bought'] = "0 osób kupiło"
    else:
        data[index]['bought'] = one.text

    parse_df_bought(index)
    parse_df_rating(index)

        

In [126]:
def get_articles(bs):

    a_id = len(df_art)

    articles = [article for article in bs.find_all('article', class_='mx7m_1 mnyp_co mlkp_ag _6a66d_YapB2', recursive=True)]  
    for article in articles:
        #ignorowanie ofert sponsorowanych
        if article.find(class_="_1y62o mpof_ki _6a66d_qjI85", recursive=True) == None:
            
            temp = article.find('a')
            offer_url = temp['href']

            details_page.append(download_details_page(offer_url, connection[1]))

            df_art.append({'article': article, 'content': details_page[-1], 'offer_url': offer_url, 'article_id': a_id})
            a_id+=1




In [122]:
def get_offer(article, page_no):
  
    offer_local_id = len(data)
 

    temp_list = article.find_all(class_="mpof_uk mp4t_0 m3h2_0 mryx_0 munh_0 mgmw_ia mg9e_0 mj7a_0 mh36_0 _6a66d_qXGzd")
    
    if temp_list[1].text == "ATX":
        temp = article["data-analytics-view-value"]
        offer_id = temp
        supply_power = temp_list[0].text
        supply_standard = temp_list[1].text
        supply_cert = temp_list[2].text
        temp = article.find('a')
        offer_url = temp['href']

        temp = article.find(class_="_1svub _lf05o")
        price = temp["aria-label"]


        data.append({'offer_local_id': int(offer_local_id),
                        'page_no': page_no,
                        'offer_id': offer_id,
                        'psu_power': supply_power,
                        'psu_standard': supply_standard,
                        'psu_cert': supply_cert,
                        'offer_url': str(offer_url),
                        'price': price,
                        'brand': None,
                        'rating': None,
                        'model': None,
                        'bought': None,
                        'votes': None
                        })

        
        parse_df_psu_power(offer_local_id)
        parse_df_price(offer_local_id)
        parse_df_psu_cert(offer_local_id)
        
        df_art.append(article)
        get_details(article)



In [3]:
def download_page(page_no, browser):
    url=f'https://allegro.pl/kategoria/podzespoly-komputerowe-zasilacze-259437?allegro-smart-standard=1&stan=nowe&certyfikat=80%20PLUS%20Bronze&certyfikat=80%20PLUS%20Silver&certyfikat=80%20PLUS%20Gold&certyfikat=80%20PLUS%20Platinum&certyfikat=80%20PLUS%20Titanium&modularne-okablowanie=w%20pełni%20modularne&modularne-okablowanie=częściowo%20modularne&modularne-okablowanie=brak&producent=Aerocool&producent=ASUS&producent=be%20quiet!&producent=Chieftec&producent=Cooler%20Master&producent=Corsair&producent=Fractal%20Design&producent=Gigabyte&producent=LC-Power&producent=LC%20Power&producent=MODECOM&producent=Seasonic&producent=SilentiumPC&producent=Thermaltake&producent=Zalman&p={page_no}'
    
    html = browser.get("view-source:" + url)
    html = browser.find_element_by_tag_name('body').text
    sleep(8)
   
    return BeautifulSoup(html)

In [113]:
def download_details_page(offer_url, browser):
    sleep(6)
    url=f'{offer_url}'
    

    html = browser.get("view-source:" + url)
    html = browser.find_element_by_tag_name('body').text
    sleep(6)
   
    return BeautifulSoup(html)

In [6]:
def get_page(page_no, browser):
    results = []
    page = download_page(page_no, browser)
    results.append({'page_no': page_no, 'content': page})

    return results

In [7]:
def is_popup(browser):
    try:
        t_button  = browser.find_element(By.XPATH, '//button[text()="Ok, zgadzam się"]')
        return t_button
    except:
        sleep(120)
        return None


In [23]:
def open_browser():
    result = []
    result.append( Options() )

    result[0].add_argument("private")
  
    browser = Opera(executable_path="./drivers/operadriver.exe",options=result[0])
    
    sleep(3)
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[1])

    sleep(3)
    browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[2])

    result.append(browser)

    return result

In [32]:
def close_browser(browser):

    sleep(3)
    browser.quit()


In [103]:


connection = open_browser()
close_browser(connection[1])

In [130]:
connection = open_browser()

data_dir = "./raw/raw_1"

df_art = []
details_page = []
data = []

page_no = 1

for entry in glob.glob(f"{data_dir}/*"):
    with open(entry, encoding="utf8") as f:
        html = f.read()
    
        bs = BeautifulSoup(html)
            
        get_articles(bs)
        
        print(page_no)
        page_no+=1




KeyboardInterrupt: 

In [114]:

df = pd.json_normalize(data, sep="_")

file = 'data_raw.csv'
path = './raw/data/'

if(os.path.exists(file) and os.path.isfile(file)):
  os.remove(file)

df.to_csv(
f"./raw/data/data_raw.csv",
sep=";",
encoding="UTF-8",
index=False
)


In [138]:
len(df_art)
print(df_art)

[{'article': <article class="mx7m_1 mnyp_co mlkp_ag _6a66d_YapB2" data-analytics-enabled="true" data-analytics-view-custom-deliverylabel="dostawa pojutrze" data-analytics-view-custom-index0="2" data-analytics-view-custom-page="1" data-analytics-view-value="9217360188" data-item="true" data-role="offer"><div class="mpof_ki mqen_m6 mp7g_oh mh36_0 mvrt_0 mg9e_8 mj7a_8 m7er_k4 _1y62o _6a66d_snEkI"><div><div class="mpof_ki myre_zn m389_6m m09p_k4 mse2_56 _6a66d_0Tggp"><div class="mpof_ki mp7g_oh"><a aria-hidden="true" class="msts_9u mg9e_0 mvrt_0 mj7a_0 mh36_0 mpof_ki m389_6m mx4z_6m m7f5_6m mse2_k4 m7er_k4 _6a66d_44ioA" href="https://allegro.pl/oferta/zasilacz-silentiumpc-vero-l3-bronze-600w-9217360188" rel="nofollow" tabindex="-1"><img alt="Zasilacz SILENTIUMPC Vero L3 Bronze 600W" class="lazyload" data-src="https://a.allegroimg.com/s180/11fdc9/d2e3ea154e0f92799e542d9817f9/Zasilacz-SILENTIUMPC-Vero-L3-Bronze-600W" src="https://assets.allegrostatic.com/metrum/placeholder/placeholder-20d85e

In [134]:

dfa = pd.json_normalize(df_art, sep="_")

file = 'raw_articles.csv'
path = './raw/data/'

if(os.path.exists(file) and os.path.isfile(file)):
    os.remove(file)


dfa.to_csv (
f"./raw/data/raw_articles.csv",
sep=";",
encoding="UTF-8",
index=False
)


RecursionError: maximum recursion depth exceeded in comparison

In [131]:
print(df.head(3))
print(df.shape)

   offer_id_local  page_no     offer_id psu_power psu_standard psu_cert  \
0               0        1   9217360188       600          ATX   Bronze   
1               1        1  11674893456       850          ATX     Gold   
2               2        1  11674681077       650          ATX   Bronze   

                                           offer_url   price        brand  \
0  https://allegro.pl/oferta/zasilacz-silentiumpc...  238,90  SilentiumPC   
1  https://allegro.pl/oferta/zasilacz-gigabyte-p8...  399,00     Gigabyte   
2  https://allegro.pl/oferta/zasilacz-corsair-cx6...  309,00      Corsair   

   rating                model offer_title bought votes  
0    4.94  Vero L3 Bronze 600W        None     73  None  
1    4.86               P850GM        None     32  None  
2    5.00        CP-9020217-EU        None      8  None  
(298, 14)


In [ ]:
for item in contents:
    with open(f"./raw/contents/{item['page_no']}_content_page.html", 'w',  encoding="utf-8") as out_file:
        out_file.write(str(item['content']))
            


In [106]:
for item in details_page:
    with open(f"./raw/contents/{offer_local_id}_details_page.html", 'w',  encoding="utf-8") as out_file:
        out_file.write(str(item))
            


NameError: name 'offer_local_id' is not defined

In [85]:
try:
    connection[1] = open_browser()

except:
    sleep(10)
    connection[1].quit
finally:
    
    df = pd.json_normalize(data, sep="_")

    file = 'raw_data.csv'
    path = './raw/data/'

    if(os.path.exists(file) and os.path.isfile(file)):
        os.remove(file)


    sleep(4)


    connection[1].get("https://twitter.pl")
    sleep(4)
    connection[1].get("https://allegro.pl/")
    sleep(4)

    button = is_popup()

    if button:
        button.click()
    else:
        connection[1].close()


    sleep(3)


    details_page = []

    for d in df.index:
        sleep(11)
        get_details(d, connection[1])

    connection[1].get("https://twitter.pl")
    sleep(4)

    #connection[1].switch_to.window(connection[1].window_handles[0])     

            
    file = 'psu_raw.csv'
    path = './raw/data/'

    if(os.path.exists(file) and os.path.isfile(file)):
        os.remove(file)
        

    close_browser(connection[1])

    df.to_csv(
    f"./raw/data/psu_raw.csv",
    sep=";",
    encoding="UTF-8",
    index=False
    )



AttributeError: 'list' object has no attribute 'get'

In [ ]:
a = 0

while a < len(details_page):
    with open(f"./raw/details_page/{a}_details_page.html", 'w',  encoding="utf-8") as out_file:
        out_file.write(str(details_page[a]))
    a+=1

NameError: name 'details_page' is not defined

In [117]:
#parsowanie

def parse_df_psu_power(index):
    temp = data[index]['psu_power']
    data[index]['psu_power'] = temp[:-2]

def parse_df_price(index):
    temp = data[index]['price'][:-18]
    data[index]['price']

def parse_df_rating(index):
    if data[index]['rating']:
        temp = str(data[index]['rating']).split('/')
        temp[0] = temp[0].replace("," , ".")
        try:
            temp[1] = str(temp[1]).split(' ')
            data[index]['rating'] = float(temp[0])
            data[index]['votes'] = temp[1][0]
        except:
            data[index]['votes'] = '?'
    else:
        data[index]['rating'] == "NaN"
        data[index]['votes'] = 0



def parse_df_bought(index):
    temp = data[index]['bought'].split(' ')
    data[index]['bought'] = temp[0]

def parse_df_psu_cert(index):
    words = [ "Bronze", "Silver", "Gold", "Platinum", "Titanium"]

    for one in words:
        if one in data[index]['psu_cert']:
            data[index]['psu_cert'] = one



    
        
    

    


In [100]:
len(df_art)

422

In [ ]:
df = pd.read_csv(
        "./raw/data/data_raw — kopia.csv",
        sep=";",
        encoding="UTF-8",
    )

votes = []

df.head(4)


for each in df.index:

    

df[] = votes



df.to_csv(
        "./raw/data/psu_raw_parsed.csv",
        sep=";",
        encoding="UTF-8",
        index= False
    )

    


C:\Users\Carra\.conda\envs\new\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Carra\.conda\envs\new\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Carra\.conda\envs\new\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Carra\.conda\envs\new\lib\site-packages\

KeyboardInterrupt: 

In [95]:
df.tail(2)



,offer_id_local,page_no,offer_id,psu_power,psu_standard,psu_cert,offer_url,price,brand,rating,model,offer_title,bought,votes
0,0,1,9217360188,,ATX,Bronze,https://allegro.pl/oferta/zasilacz-silentiumpc...,"238,90",SilentiumPC,4.94,Vero L3 Bronze 600W,Zasilacz SilentiumPC Vero L3 Bronze 600 W,73,None


In [39]:
import random

df = pd.read_csv(
        "./raw/data/data_raw.csv",
        sep=";",
        encoding="UTF-8",
    )

s = []

for i in df.index:
    s.append(random.randint(0,301))

df['bought']= pd.Series(s)



df.to_csv(
        "./raw/data/data_raw — kopia.csv",
        sep=";",
        encoding="UTF-8",
        index= False
    )
